In [ ]:
!pip install langchain langchain-google-genai chromadb google-generativeai

In [2]:
from langchain.schema import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

docs_travel = [
    Document(
        page_content="Travories is a travel booking platform where travel agencies can create and list their tour packages. Tourists can browse and book these packages directly through the platform. Travories aims to connect travelers with a wide variety of curated travel experiences.",
        metadata={"source": "overview.txt"}
    ),
    Document(
        page_content="Travories is accessible 24 hours a day, 7 days a week, allowing both agencies and tourists to use the platform at their convenience, regardless of their time zone.",
        metadata={"source": "availability.txt"}
    ),
    Document(
        page_content="Travel agencies on Travories have the ability to create detailed descriptions of their tour packages, including itineraries, pricing, inclusions, and stunning visuals. They manage their own listings and availability.",
        metadata={"source": "agency_features.txt"}
    ),
    Document(
        page_content="Tourists using Travories can search for travel packages based on destination, interests, duration, and budget. They can view package details, read reviews (if available), and securely book their chosen tours through the platform.",
        metadata={"source": "tourist_features.txt"}
    ),
    Document(
        page_content="For any inquiries or support, users can contact Travories customer support via email at support@travories.com or through the live chat feature available on the website and mobile app.",
        metadata={"source": "support.txt"}
    ),
    Document(
        page_content="Travories supports various payment methods, including major credit cards, debit cards, and popular online payment gateways, ensuring a seamless booking experience for tourists worldwide.",
        metadata={"source": "payment.txt"}
    ),
]

# Create the Chroma vector store using Gemini embeddings
db_gemini = Chroma.from_documents(docs_travel, gemini_embeddings)

/Users/prajwol/Documents/travories-geolocation/LangGraph/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
retriever = db_gemini.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [4]:
from langchain_core.prompts import ChatPromptTemplate

template_travel = """
Answer the question based only on the following context: {context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template_travel)
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer the question based only on the following context: {context}\nQuestion: {question}\n'), additional_kwargs={})]


In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain_travel = (
    {
        "context": lambda x: format_docs(retriever.invoke(x)),
        "question": lambda x: x
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [6]:
print(qa_chain_travel.invoke("What is Travories and how does it work?"))
print("\n---")
print(qa_chain_travel.invoke("Can travel agencies list their packages?"))
print("\n---")
print(qa_chain_travel.invoke("What are the ways tourists can pay?"))
print("\n---")
print(qa_chain_travel.invoke("Is Travories open all the time?"))

Travories is a travel booking platform connecting travelers with a wide variety of curated travel experiences.  Travel agencies list their tour packages on the platform, which tourists can then browse and book.  Tourists can search for packages based on destination, interests, duration, and budget, view details and reviews, and make secure bookings using various payment methods.  The platform is accessible 24/7.

---
Yes, the text states that travel agencies on Travories can create and list their tour packages.

---
Tourists can pay using major credit cards, debit cards, and popular online payment gateways.

---
Yes, Travories is accessible 24 hours a day, 7 days a week.
